In [309]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import TeamEstimatedMetrics
from nba_api.stats.endpoints import PlayerEstimatedMetrics
import pandas as pd
import numpy as np
import json
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [307]:
# Nikola Jokić
id = '203999'

team_id_map = {
    "ATL": 1610612737, "BOS": 1610612738, "BKN": 1610612751, "CHA": 1610612766,
    "CHI": 1610612741, "CLE": 1610612739, "DAL": 1610612742, "DEN": 1610612743,
    "DET": 1610612765, "GSW": 1610612744, "HOU": 1610612745, "IND": 1610612754,
    "LAC": 1610612746, "LAL": 1610612747, "MEM": 1610612763, "MIA": 1610612748,
    "MIL": 1610612749, "MIN": 1610612750, "NOP": 1610612740, "NYK": 1610612752,
    "OKC": 1610612760, "ORL": 1610612753, "PHI": 1610612755, "PHX": 1610612756,
    "POR": 1610612757, "SAC": 1610612758, "SAS": 1610612759, "TOR": 1610612761,
    "UTA": 1610612762, "WAS": 1610612764
}

season_team_def_rating = {}

for start_year in range(2003, 2015):
    season = f"{start_year}-{str(start_year+1)[-2:]}"
    team_metrics_df = TeamEstimatedMetrics(
        league_id = '00',
        season = season,
        season_type = 'Regular Season'
    ).get_data_frames()[0]
    team_def_dict = dict(zip(team_metrics_df["TEAM_ID"], team_metrics_df["E_DEF_RATING"]))
    season_team_def_rating[season] = team_def_dict


In [337]:
for start_year in range(2015, 2025):
    season = f"{start_year}-{str(start_year+1)[-2:]}"
    team_metrics_df = TeamEstimatedMetrics(
        league_id = '00',
        season = season,
        season_type = 'Regular Season'
    ).get_data_frames()[0]
    team_def_dict = dict(zip(team_metrics_df["TEAM_ID"], team_metrics_df["E_DEF_RATING"]))
    season_team_def_rating[season] = team_def_dict

In [338]:
with open("def_ratings.json", "w") as json_file:
    json.dump(season_team_def_rating, json_file)

In [339]:
career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0]
years = career['SEASON_ID'].unique()
games = []

for yr in years:
    season = pd.DataFrame(playergamelog.PlayerGameLog(player_id='203999', season = yr, season_type_all_star = 'Regular Season').get_data_frames()[0]).iloc[::-1]
    games.append(season)

games = pd.concat(games, ignore_index=True)
games['MATCHUP'] = games['MATCHUP'].str[-3:]
games = games.drop('VIDEO_AVAILABLE', axis = 1)
print(games)

    SEASON_ID  Player_ID     Game_ID     GAME_DATE MATCHUP WL  MIN  FGM  FGA  \
0       22015     203999  0021500010  OCT 28, 2015     HOU  W    4    1    1   
1       22015     203999  0021500029  OCT 30, 2015     MIN  L   18    5    8   
2       22015     203999  0021500044  NOV 01, 2015     OKC  L   24    4    9   
3       22015     203999  0021500073  NOV 05, 2015     UTA  L   18    1    4   
4       22015     203999  0021500083  NOV 06, 2015     GSW  L   17    2    4   
..        ...        ...         ...           ...     ... ..  ...  ...  ...   
721     22024     203999  0022400749  FEB 08, 2025     PHX  W   29   11   13   
722     22024     203999  0022400763  FEB 10, 2025     POR  W   31   15   22   
723     22024     203999  0022400780  FEB 12, 2025     POR  W   36   11   20   
724     22024     203999  0022400794  FEB 20, 2025     CHA  W   37   11   22   
725     22024     203999  0022400808  FEB 22, 2025     LAL  L   35    2    7   

     FG_PCT  ...  OREB  DREB  REB  AST 

# Now we have games df with every single game in the player's history
### Need to extract: 

- [x] Days between Game
- [x] Average over L5 Games 
- [x] Average over L10 Games
- [x] Average L5 vs Current Opponent
- [x] Average L2 vs Current Opponent
- [x] Player Age
- [x] L5 Plus/Minus
- [x] Opposing Team's Defensive Rating (For the season)
- [x] Average Minutes Played over L5
- [ ] Home/Away Game


In [340]:
#Average over L5 Games
games['L5'] = games['PTS'].shift(1).rolling(5).mean()
print(games['L5'].iloc[5:])

5       5.8
6       7.4
7       7.0
8       6.0
9       5.8
       ... 
721    29.8
722    29.4
723    31.8
724    31.6
725    29.8
Name: L5, Length: 721, dtype: float64


In [341]:
#Average over L10 Games
games['L10'] = games['PTS'].shift(1).rolling(10).mean()
print(games['L10'].iloc[10:])

10      5.4
11      7.5
12      7.1
13      7.4
14      8.5
       ... 
721    28.1
722    28.0
723    28.5
724    29.1
725    28.7
Name: L10, Length: 716, dtype: float64


In [342]:
#Average over L5 vs Opponent
teams = games['MATCHUP'].unique()

for team in teams:
    temp = games[games['MATCHUP'].str.contains(team)]['PTS'].shift(1).rolling(5).mean()
    for i in range(len(temp)):
        games.loc[games[games['MATCHUP'].str.contains(team)].index[i], 'L5 vs OPP'] = temp.iloc[i]
        
games['L5 vs OPP'].fillna(games['L5'], inplace=True)

print(games['L5 vs OPP'].iloc[5:])

5       5.8
6       7.4
7       7.0
8       6.0
9       5.8
       ... 
721    25.0
722    31.0
723    31.8
724    25.8
725    27.2
Name: L5 vs OPP, Length: 721, dtype: float64


In [343]:
#Average over L5 vs Opponent
teams = games['MATCHUP'].unique()

for team in teams:
    temp = games[games['MATCHUP'].str.contains(team)]['PTS'].shift(1).rolling(2).mean()
    for i in range(len(temp)):
        games.loc[games[games['MATCHUP'].str.contains(team)].index[i], 'L2 vs OPP'] = temp.iloc[i]
        
games['L2 vs OPP'].fillna(games['L5'], inplace=True)

print(games['L2 vs OPP'].iloc[5:])

5       5.8
6       7.4
7       7.0
8       6.0
9       5.8
       ... 
721    28.5
722    31.5
723    37.0
724    20.5
725    34.5
Name: L2 vs OPP, Length: 721, dtype: float64


In [344]:
#Days between Game
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'], format="%b %d, %Y")
games['Days Off'] = games['GAME_DATE'].diff().dt.days

print(games['Days Off'].iloc[1:])

1      2.0
2      2.0
3      4.0
4      1.0
5      3.0
      ... 
721    2.0
722    2.0
723    2.0
724    8.0
725    2.0
Name: Days Off, Length: 725, dtype: float64


In [345]:
#Age
player_info = commonplayerinfo.CommonPlayerInfo(player_id=id)
birthday = datetime.strptime(player_info.get_data_frames()[0]['BIRTHDATE'][0], "%Y-%m-%dT%H:%M:%S")
date = datetime.today()
age = date.year - birthday.year - ((date.month, date.day) < (birthday.month, birthday.day))
games['Age'] = (games['GAME_DATE'].dt.year - birthday.year) - \
               ((games['GAME_DATE'].dt.month < birthday.month) | 
                ((games['GAME_DATE'].dt.month == birthday.month) & (games['GAME_DATE'].dt.day < birthday.day)))
print(games['Age'])

0      20
1      20
2      20
3      20
4      20
       ..
721    29
722    29
723    29
724    30
725    30
Name: Age, Length: 726, dtype: int64


In [346]:
#L5 Plus/Minus
games['L5 Plus/Minus'] = games['PLUS_MINUS'].shift(1).rolling(5).mean()
print(games['L5 Plus/Minus'].iloc[5:])

5      -0.4
6      -0.8
7      -1.0
8       0.4
9       2.0
       ... 
721    11.0
722    14.8
723    19.6
724    18.6
725    16.6
Name: L5 Plus/Minus, Length: 721, dtype: float64


In [347]:
#Opposing Team's Defense Rating
games["MATCHUP_ID"] = games["MATCHUP"].map(team_id_map)
def get_opponent_def_rating(row):
    season = int(row["SEASON_ID"][-4:])
    year = f"{season}-{str(season+1)[-2:]}"
    opponent_team_abbr = row["MATCHUP"] 
    opponent_team_id = team_id_map[opponent_team_abbr]
    
    if opponent_team_id is None:
        return None
    
    return season_team_def_rating.get(year, {}).get(opponent_team_id, None)

games["OPP_DEF_RATING"] = games.apply(get_opponent_def_rating, axis=1) 

In [348]:
print(games)

    SEASON_ID  Player_ID     Game_ID  GAME_DATE MATCHUP WL  MIN  FGM  FGA  \
0       22015     203999  0021500010 2015-10-28     HOU  W    4    1    1   
1       22015     203999  0021500029 2015-10-30     MIN  L   18    5    8   
2       22015     203999  0021500044 2015-11-01     OKC  L   24    4    9   
3       22015     203999  0021500073 2015-11-05     UTA  L   18    1    4   
4       22015     203999  0021500083 2015-11-06     GSW  L   17    2    4   
..        ...        ...         ...        ...     ... ..  ...  ...  ...   
721     22024     203999  0022400749 2025-02-08     PHX  W   29   11   13   
722     22024     203999  0022400763 2025-02-10     POR  W   31   15   22   
723     22024     203999  0022400780 2025-02-12     POR  W   36   11   20   
724     22024     203999  0022400794 2025-02-20     CHA  W   37   11   22   
725     22024     203999  0022400808 2025-02-22     LAL  L   35    2    7   

     FG_PCT  ...  PLUS_MINUS    L5   L10  L5 vs OPP  L2 vs OPP  Days Off  A

In [349]:
#L5 Min Played
games['L5 Min Played'] = games['MIN'].shift(1).rolling(5).mean()
print(games['L5 Min Played'].iloc[5:])

5      16.2
6      18.6
7      17.8
8      14.2
9      14.2
       ... 
721    35.4
722    33.8
723    32.6
724    32.6
725    32.8
Name: L5 Min Played, Length: 721, dtype: float64


In [350]:
games['ACTUAL PTS'] = games['PTS']
print(games)

    SEASON_ID  Player_ID     Game_ID  GAME_DATE MATCHUP WL  MIN  FGM  FGA  \
0       22015     203999  0021500010 2015-10-28     HOU  W    4    1    1   
1       22015     203999  0021500029 2015-10-30     MIN  L   18    5    8   
2       22015     203999  0021500044 2015-11-01     OKC  L   24    4    9   
3       22015     203999  0021500073 2015-11-05     UTA  L   18    1    4   
4       22015     203999  0021500083 2015-11-06     GSW  L   17    2    4   
..        ...        ...         ...        ...     ... ..  ...  ...  ...   
721     22024     203999  0022400749 2025-02-08     PHX  W   29   11   13   
722     22024     203999  0022400763 2025-02-10     POR  W   31   15   22   
723     22024     203999  0022400780 2025-02-12     POR  W   36   11   20   
724     22024     203999  0022400794 2025-02-20     CHA  W   37   11   22   
725     22024     203999  0022400808 2025-02-22     LAL  L   35    2    7   

     FG_PCT  ...   L10  L5 vs OPP  L2 vs OPP  Days Off  Age  L5 Plus/Minus 

In [351]:
data = games[['L5', 'L10', 'L5 vs OPP', 'L2 vs OPP', 'Days Off', 'Age', 'L5 Plus/Minus', 'OPP_DEF_RATING', 'L5 Min Played', 'ACTUAL PTS']].iloc[10:].to_numpy()
np.savetxt("jokic.txt", data, fmt = "%.2f")
#indexes: 0 = L5, 1 = L10, 2 = L5 vs OPP, 3 = L2 vs OPP, 4 = Days Off, 5 = Age, 6 = L5 +/-, 7 = OPP_DEF_RATING, 8 = L5 Min Played, 9 = Actual Pts

In [352]:
X = data[:, :-1]
y = data[:, -1]
x_scaler = MinMaxScaler()
y_scaler = StandardScaler()
X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1, 1))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [353]:
nn_model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1)
])

nn_model.compile(optimizer="adam", loss="mse", metrics=["mae"])
nn_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=16)

Epoch 1/100


/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1796 - mae: 0.8701 - val_loss: 0.8970 - val_mae: 0.7747
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9208 - mae: 0.7605 - val_loss: 0.8195 - val_mae: 0.7388
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8819 - mae: 0.7417 - val_loss: 0.7626 - val_mae: 0.7089
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.8086 - mae: 0.7009 - val_loss: 0.7274 - val_mae: 0.6893
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7493 - mae: 0.6713 - val_loss: 0.6896 - val_mae: 0.6688
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7855 - mae: 0.6964 - val_loss: 0.6779 - val_mae: 0.6612
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6608 - mae: 0.6288 - val_loss: 0.6602 - val_mae: 0.6500
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7711 - mae: 0.6999 - val_loss: 0.6580 - val_mae: 0.6482
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7639 - mae: 0.6

36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7273 - mae: 0.6612 - val_loss: 0.6269 - val_mae: 0.6160
Epoch 56/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6926 - mae: 0.6355 - val_loss: 0.6256 - val_mae: 0.6164
Epoch 57/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7120 - mae: 0.6493 - val_loss: 0.6218 - val_mae: 0.6184
Epoch 58/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6676 - mae: 0.6301 - val_loss: 0.6256 - val_mae: 0.6205
Epoch 59/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6259 - mae: 0.6248 - val_loss: 0.6223 - val_mae: 0.6155
Epoch 60/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6765 - mae: 0.6267 - val_loss: 0.6311 - val_mae: 0.6277
Epoch 61/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.7653 - mae: 0.6848 - val_loss: 0.6253 - val_mae: 0.6159
Epoch 62/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6337 - mae: 0.6156 - val_loss: 0.6237 - val_mae: 0.6187
Epoch 63/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6845 - 

In [355]:
nn_preds = nn_model.predict(X_test)
print("Neural Network MAE:", mean_absolute_error(y_test, nn_preds))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Neural Network MAE: 0.6120898419082521


In [356]:
print(nn_preds)

[[-0.2830026 ]
 [ 0.010989  ]
 [-0.9229018 ]
 [ 0.68527085]
 [ 0.44727653]
 [ 0.30825943]
 [-1.2326698 ]
 [ 0.20194852]
 [-1.0637805 ]
 [ 0.50506437]
 [ 0.20280805]
 [-0.7344735 ]
 [ 0.54520166]
 [-1.1032722 ]
 [-0.14944547]
 [ 0.20174044]
 [-0.4111923 ]
 [-0.14257869]
 [ 0.48718697]
 [ 0.2996055 ]
 [-1.0757835 ]
 [-0.27873388]
 [ 0.7759891 ]
 [ 0.4364168 ]
 [-0.5826617 ]
 [-0.37419888]
 [-0.35772088]
 [-0.8300799 ]
 [-1.2461784 ]
 [ 0.01269075]
 [ 0.10240488]
 [-0.3323463 ]
 [-0.03478024]
 [ 0.30148855]
 [-1.4038    ]
 [-0.04605623]
 [-1.034023  ]
 [ 0.43868893]
 [-0.36228076]
 [ 0.4249397 ]
 [-0.316182  ]
 [ 0.1139387 ]
 [ 0.46249008]
 [-0.10221811]
 [-0.08847559]
 [-1.1222792 ]
 [ 0.15192306]
 [ 0.5873544 ]
 [-0.10262202]
 [ 0.10149505]
 [-0.42847982]
 [-0.58953744]
 [ 0.60095775]
 [-0.28942844]
 [ 0.33110917]
 [ 0.2770491 ]
 [-1.2569292 ]
 [-0.37419492]
 [ 0.13417435]
 [ 0.17518166]
 [-1.4610108 ]
 [ 0.12670025]
 [ 0.24634829]
 [ 0.69157076]
 [-0.53358334]
 [ 0.24264705]
 [ 0.41676

In [357]:
y_pred = y_scaler.inverse_transform(nn_preds)

In [358]:
y_prev = y_scaler.inverse_transform(y_test)

In [359]:
comparison = np.hstack((y_prev.reshape(-1,1), y_pred))

In [360]:
print(comparison)

[[ 2.         19.029459  ]
 [22.         21.8715992 ]
 [13.         12.84328842]
 [23.         28.39016151]
 [38.         26.08937073]
 [19.         24.74543571]
 [ 6.          9.84863186]
 [32.         23.71768379]
 [ 6.         11.48135471]
 [29.         26.64802933]
 [21.         23.7259922 ]
 [24.         14.66490269]
 [35.         27.0360527 ]
 [ 9.         11.09957218]
 [25.         20.32061195]
 [26.         23.71567154]
 [15.         17.79019737]
 [24.         20.38699532]
 [40.         26.47520065]
 [25.         24.66177368]
 [12.         11.36531734]
 [14.         19.07072639]
 [56.         29.26717186]
 [27.         25.98438454]
 [14.         16.13253021]
 [16.         18.14782715]
 [ 8.         18.307127  ]
 [12.         13.74063587]
 [13.          9.71803951]
 [35.         21.88805008]
 [23.         22.75535393]
 [15.         18.55243301]
 [19.         21.42912865]
 [26.         24.67997742]
 [ 8.          8.19424534]
 [30.         21.32011795]
 [ 4.         11.76903248]
 

In [382]:
ans = 0
rows = 0
for row in comparison:
    if row[1] > 25 and row[0] > 25:
        ans = ans + 1
    if row[1] < 25 and row[0] < 25:
        ans = ans + 1
    
print(ans)

98
